In [ ]:
!pip install pillow transformers torch requests beautifulsoup4 google-search-results
!pip install openai
!pip install python-dotenv

  Preparing metadata (setup.py) ... done
  Created wheel for google-search-results: filename=google_search_results-2.4.2-py3-none-any.whl size=32010 sha256=c01d7664f40e4f999f6ce63cc8c4b3218a31cc474336129d95ea174b7b4acccd
  Stored in directory: /root/.cache/pip/wheels/d3/b2/c3/03302d12bb44a2cdff3c9371f31b72c0c4e84b8d2285eeac53
Successfully built google-search-results
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.7/383.7 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.2/325.2 kB 22.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.6 MB/s eta 0:00:00


In [ ]:
import random
import textwrap
import requests
from PIL import Image, ImageDraw, ImageFont
from transformers import pipeline
from io import BytesIO
from bs4 import BeautifulSoup


generator = pipeline('text-generation', model='gpt2')

def generate_caption(prompt="funny meme about"):
    caption = generator(prompt, max_length=30, num_return_sequences=1)[0]['generated_text']
    return caption.split('.')[0]

def download_image(search_query):
    search_url = f"https://www.google.com/search?tbm=isch&q={search_query}"
    headers = {
        "User-Agent": "Mozilla/5.0"
    }


    response = requests.get(search_url, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')

    image_elements = soup.find_all('img')
    image_urls = [img['src'] for img in image_elements if 'src' in img.attrs]

    if not image_urls:
        raise Exception("No images found for the search query.")

    random_image_url = random.choice(image_urls)

    image_response = requests.get(random_image_url)
    img = Image.open(BytesIO(image_response.content))

    return img

def add_caption_to_image(img, caption, output_path):
    draw = ImageDraw.Draw(img)


    try:
        font = ImageFont.truetype("arial.ttf", 40)
    except IOError:
        font = ImageFont.load_default()


    margin = 10
    image_width = img.size[0]
    lines = textwrap.wrap(caption, width=40)


    y_text = img.size[1] - len(lines) * 45 - 20


    for line in lines:
        bbox = draw.textbbox((0, 0), line, font=font)
        width = bbox[2] - bbox[0]
        height = bbox[3] - bbox[1]
        draw.text(((image_width - width) / 2, y_text), line, font=font, fill="white", stroke_fill="black", stroke_width=2)
        y_text += height


    img.save(output_path)


def generate_meme(search_query, output_path, prompt="funny meme about"):

    img = download_image(search_query)


    caption = generate_caption(prompt)


    add_caption_to_image(img, caption, output_path)

    print(f"Meme generated and saved to {output_path}")


if __name__ == "__main__":

    search_query = input("Enter the search query for the image: ")
    prompt = input("Enter the prompt for the meme: ")


    output_path = "generated_meme.jpg"


    generate_meme(search_query, output_path, prompt= prompt)


Enter the search query for the image: sunny leone
Enter the prompt for the meme: i love sex


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Meme generated and saved to generated_meme.jpg
